<a href="https://colab.research.google.com/github/Mye0n9/AAI_Pitcher/blob/CNN_Testing/Pitcher_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import glob
import cv2
import re
import random
from google.colab.patches import cv2_imshow


print("NumPy Version :{}".format(np.__version__))
print("TensorFlow Version :{}".format(tf.__version__))
print("Matplotlib Version :{}".format(plt.matplotlib.__version__))

# **DATA_PREPROCESSING**

In [ ]:
image_data = []
label_data = []
for i in range(1,9):
  for j in range(1,33):
    image_link = '/content/drive/MyDrive/Raw_Data/{one}/{two}/*'.format(one = i,two = j)
    label_link = '/content/drive/MyDrive/Label/{one}/{two}/*'.format(one = i,two = j)
    image_data.append(glob.glob(image_link))
    label_data.append(glob.glob(label_link))


[['/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00001.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00018.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00028.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00014.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00026.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00012.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00009.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00010.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00021.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00027.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00006.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00023.jpg', '/content/drive/MyDrive/Raw_Data/1/1/baseball_ra_po_d02_00001_00015.jpg', '/content/drive/MyDrive/Raw_Data/1/1

In [ ]:
def cropper (img, point, num1, num2, size):
  # num1 is x
  # num2 is y
  max_x = 1920
  max_y = 1080

  x,y = point[num1], point[num2]
  x1,x2,y1,y2 = 0,0,0,0

  # x1 is smaller than x2
  # y1 is smaller than y1

  if point[num1]+size > max_x :
    x1 = point[num1] - size*2
    x2 = point[num1]
  else:
    x1 = point[num1]-size
    x2 = point[num2]+size
  
  if point[num2] + size > max_y:
    y1 = point[num2] - size
    y2 = point[num2]
  else:
    y1 = point[num2] - size
    y2 = point[num2] + size


  cropped_img = img[y1:y2, x1:x2]
  return cropped_img

In [ ]:
import json # 나중에 올리기
shoulders = []
elbows = []
wrists = []
IMG_SIZE = 64
for i in range(len(image_data)):
  for j in range(len(image_data[i])):
    f = open(label_data[i][j])
    img = cv2.imread(image_data[i][j],cv2.IMREAD_COLOR)
    data = json.load(f)
    points = data['annotations'][1]['points']
    # shoulder (6,7 -> 15~20)
    shoulder_point = points[15:21]
    shoulders.append(cropper(img,shoulder_point,0,1,int(IMG_SIZE/2)))
    shoulders.append(cropper(img,shoulder_point,3,4,int(IMG_SIZE/2)))
    # elbows (8,9 -> 21~27)
    elbow_point = points[21:27]
    elbows.append(cropper(img,elbow_point,0,1,int(IMG_SIZE/2)))
    elbows.append(cropper(img,elbow_point,3,4,int(IMG_SIZE/2)))
    # wrists (10,11 -> 27~33)
    wrist_point = points[27:33]
    wrists.append(cropper(img,wrist_point,0,1,int(IMG_SIZE/2)))
    wrists.append(cropper(img,wrist_point,3,4,int(IMG_SIZE/2)))
    f.close()
print(wrists[0])